In [19]:
%pip install numpy pandas joblib scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import numpy as np
import pandas as pd
import joblib
import json
from pathlib import Path
from typing import Dict, Any
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import scipy.sparse as sp

In [21]:
# Definisi RuleFitLite (harus ada sebelum load model!)
class RuleFitLite:
    def __init__(self, n_estimators=200, max_depth=5, min_samples_leaf=1,
                 random_state=42, alpha=1.0):
        self.rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            random_state=random_state,
            n_jobs=-1,
        )
        self.alpha = float(alpha)
        self.lr = LogisticRegression(
            penalty="l1",
            solver="saga",
            C=1.0/max(self.alpha, 1e-8),
            max_iter=3000,
            multi_class="auto",
        )
        self.classes_ = None

    def _rules_matrix(self, X):
        mats = []
        for est in self.rf.estimators_:
            M = est.decision_path(X)
            if M.shape[1] > 1:
                M = M[:, 1:]
            mats.append(M)
        if len(mats) == 1:
            return mats[0].tocsr()
        return sp.hstack(mats, format="csr")

    def fit(self, X, y):
        X = np.asarray(X)
        self.rf.fit(X, y)
        self.classes_ = np.unique(y)
        R = self._rules_matrix(X)
        self.lr.fit(R, y)
        return self

    def predict_proba(self, X):
        X = np.asarray(X)
        R = self._rules_matrix(X)
        return self.lr.predict_proba(R)

    def predict(self, X):
        probs = self.predict_proba(X)
        idx = probs.argmax(axis=1)
        return np.array([self.lr.classes_[i] for i in idx])

print("✅ RuleFitLite class loaded")

✅ RuleFitLite class loaded


In [ ]:
ARTIFACTS_DIR = Path("artifacts")

# Load semua artifacts yang diperlukan
with open(ARTIFACTS_DIR / "feature_columns.json", "r") as f:
    feature_columns = json.load(f)

with open(ARTIFACTS_DIR / "best_model_name.json", "r") as f:
    best_model_info = json.load(f)
    best_model_name = best_model_info["best_model"]

# Load KEDUA model
model_dt = joblib.load(ARTIFACTS_DIR / "job_predictor_dt_normalized.pth")
model_rf = joblib.load(ARTIFACTS_DIR / "job_predictor_rulefit_normalized.pth")
preprocess = joblib.load(ARTIFACTS_DIR / "preprocess_normalized.pth")

print("✅ Loaded: Decision Tree")
print("✅ Loaded: RuleFit-Lite")
print(f"\nTotal fitur: {len(feature_columns)}")
print(f"Best Model: {best_model_name}")

✅ Loaded: Decision Tree
✅ Loaded: RuleFit-Lite

Total fitur: 27
Best Model: RuleFit-Lite


In [23]:
def get_user_input(feature_columns):
    print("="*60)
    print("MASUKKAN NILAI SKILL KAMU")
    print("="*60)
    print("Tips: Masukkan nilai 1-5 untuk setiap skill\n")
    
    data = {}
    
    for ft in feature_columns:
        while True:
            try:
                v = input(f"{ft} (1-5): ")
                if v.strip() == "":  # Skip jika kosong
                    data[ft] = np.nan
                    break
                    
                v = float(v)
                if 1 <= v <= 5:
                    data[ft] = v
                    break
                else:
                    print("⚠️  Masukkan angka 1-5!")
            except:
                print("⚠️  Input tidak valid!")
    
    return data

# Input dari user
user_input = get_user_input(feature_columns)

# Convert ke DataFrame (tidak perlu binning lagi!)
user_df = pd.DataFrame([user_input])

print("\n" + "="*60)
print("DATA INPUT USER")
print("="*60)
print(user_df.head())

MASUKKAN NILAI SKILL KAMU
Tips: Masukkan nilai 1-5 untuk setiap skill


DATA INPUT USER
   Database Fundamentals  Computer Architecture  \
0                    5.0                    5.0   

   Distributed Computing Systems  Cyber Security  Networking  \
0                            5.0             5.0         5.0   

   Software Development  Programming Skills  Project Management  \
0                   5.0                 5.0                 5.0   

   Computer Forensics Fundamentals  Technical Communication  ...  Openness  \
0                              5.0                      5.0  ...       5.0   

   Conscientousness  Extraversion  Agreeableness  Emotional_Range  \
0               5.0           5.0            5.0              5.0   

   Conversation  Openness to Change  Hedonism  Self-enhancement  \
0           5.0                 5.0       5.0               5.0   

   Self-transcendence  
0                 5.0  

[1 rows x 27 columns]


In [24]:
def predict_with_model(user_data: Dict[str, float], model, model_name: str, is_rulefit: bool = False):
    # Hitung overall skill score
    skill_scores = [float(v) for v in user_data.values() if not pd.isna(v)]
    avg_skill = np.mean(skill_scores) if skill_scores else 0
    
    # Prediksi
    user_df = pd.DataFrame([user_data])
    
    if is_rulefit:
        user_df_pre = preprocess.transform(user_df)
        prediction = model.predict(user_df_pre)[0]
        try:
            proba = model.predict_proba(user_df_pre)[0]
            classes = model.lr.classes_
        except:
            proba, classes = None, None
    else:  # Decision Tree
        prediction = model.predict(user_df)[0]
        try:
            proba = model.predict_proba(user_df)[0]
            classes = model.classes_
        except:
            proba, classes = None, None
    
    # Hasil
    result = {
        "model": model_name,
        "prediction": str(prediction),
        "average_skill_score": round(avg_skill, 2),
        "rating": "⭐" * int(avg_skill)
    }
    
    if proba is not None and classes is not None:
        order = np.argsort(proba)[::-1]
        result["top_recommendations"] = [
            {"role": str(classes[i]), "probability": round(float(proba[i]), 4)}
            for i in order[:5]
        ]
    
    return result

# Prediksi dengan KEDUA model
result_dt = predict_with_model(user_input, model_dt, "Decision Tree", is_rulefit=False)
result_rf = predict_with_model(user_input, model_rf, "RuleFit-Lite", is_rulefit=True)

print("\n" + "="*80)
print("HASIL PREDIKSI - RULEFIT-LITE")
print("="*80)
print(f"\n🎯 Job Terbaik: {result_rf['prediction']}")
print(f"📊 Skill Score: {result_rf['average_skill_score']}/5 {result_rf['rating']}\n")

if "top_recommendations" in result_rf:
    print("📋 Top 5 Rekomendasi:")
    for i, rec in enumerate(result_rf["top_recommendations"], 1):
        print(f"  {i}. {rec['role']} ({rec['probability']*100:.1f}%)")
        
print("\n" + "="*80)
print("HASIL PREDIKSI - DECISION TREE")
print("="*80)
print(f"\n🎯 Job Terbaik: {result_dt['prediction']}")
print(f"📊 Skill Score: {result_dt['average_skill_score']}/5 {result_dt['rating']}\n")

if "top_recommendations" in result_dt:
    print("📋 Top 5 Rekomendasi:")
    for i, rec in enumerate(result_dt["top_recommendations"], 1):
        print(f"  {i}. {rec['role']} ({rec['probability']*100:.1f}%)")


# Perbandingan
print("\n" + "="*80)
print("PERBANDINGAN")
print("="*80)
print(f"Decision Tree    → {result_dt['prediction']}")
print(f"RuleFit-Lite     → {result_rf['prediction']}")
if result_dt['prediction'] == result_rf['prediction']:
    print("✅ Kedua model memberikan prediksi yang SAMA!")
else:
    print("⚠️  Kedua model memberikan prediksi yang BERBEDA")


HASIL PREDIKSI - RULEFIT-LITE

🎯 Job Terbaik: Customer Service Executive
📊 Skill Score: 5.0/5 ⭐⭐⭐⭐⭐

📋 Top 5 Rekomendasi:
  1. Customer Service Executive (10.4%)
  2. Information Security Specialist (10.4%)
  3. Networking Engineer (9.8%)
  4. Software Developer (8.1%)
  5. Cyber Security Specialist (7.7%)

HASIL PREDIKSI - DECISION TREE

🎯 Job Terbaik: Customer Service Executive
📊 Skill Score: 5.0/5 ⭐⭐⭐⭐⭐

📋 Top 5 Rekomendasi:
  1. Customer Service Executive (100.0%)
  2. Technical Writer (0.0%)
  3. Software Developer (0.0%)
  4. Software tester (0.0%)
  5. Networking Engineer (0.0%)

PERBANDINGAN
Decision Tree    → Customer Service Executive
RuleFit-Lite     → Customer Service Executive
✅ Kedua model memberikan prediksi yang SAMA!
